![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FFramework+Workflows%2FPyTorch%2Fserving&file=dataflow-streaming-runinference.ipynb)
<!--- header table --->
<table align="left">
<tr>
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FFramework%2520Workflows%2FPyTorch%2Fserving%2Fdataflow-streaming-runinference.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery logo">
      <br>Open in<br>BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Framework%20Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</tr>
<tr>
  <td colspan="5" style="text-align: right">
    <b>Share This On: </b> 
    <a href="https://www.linkedin.com/sharing/share-offsite/?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="Linkedin Logo" width="20px"></a> 
    <a href="https://reddit.com/submit?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb"><img src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit Logo" width="20px"></a> 
    <a href="https://bsky.app/intent/compose?text=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="BlueSky Logo" width="20px"></a> 
    <a href="https://twitter.com/intent/tweet?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X (Twitter) Logo" width="20px"></a> 
  </td>
</tr>
<tr>
  <td colspan="5" style="text-align: right">
    <b>Connect With Author On: </b> 
    <a href="https://www.linkedin.com/in/statmike"><img src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="Linkedin Logo" width="20px"></a>
    <a href="https://www.github.com/statmike"><img src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub Logo" width="20px"></a> 
    <a href="https://www.youtube.com/@statmike-channel"><img src="https://upload.wikimedia.org/wikipedia/commons/f/fd/YouTube_full-color_icon_%282024%29.svg" alt="YouTube Logo" width="20px"></a>
    <a href="https://bsky.app/profile/statmike.bsky.social"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="BlueSky Logo" width="20px"></a> 
    <a href="https://x.com/statmike"><img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X (Twitter) Logo" width="20px"></a>
  </td>
</tr>
</table>

# Dataflow Streaming Inference with RunInference

This notebook demonstrates streaming processing of transactions using Dataflow with Apache Beam RunInference.

## What You'll Learn

This workflow covers:

1. **Create Custom ModelHandler**: Wrap PyTorch model for RunInference
2. **Build Streaming Pipeline**: Read from Pub/Sub, apply model, write to BigQuery and Pub/Sub
3. **Run on Dataflow**: Execute continuous pipeline on Google Cloud
4. **Monitor Job**: Track streaming job progress and view results
5. **Clean Up**: Stop streaming job to avoid ongoing charges

## Prerequisites

- Completed `dataflow-setup.ipynb` - This sets up:
  - Model .pt file extracted from .mar and uploaded to GCS
  - BigQuery tables created (including `pytorch_autoencoder_streaming_results`)
  - Pub/Sub topics and subscriptions created

## Batch vs Streaming

**Batch Processing (Previous Notebook)**:
- ✅ Process historical data
- ✅ Bounded dataset (has a start and end)
- ✅ Results available when job completes
- ✅ Cost-effective for large datasets
- Example: Analyze all transactions from last month

**Streaming Processing (This Notebook)**:
- ✅ Process real-time data
- ✅ Unbounded dataset (continuous)
- ✅ Results available immediately
- ✅ Low-latency anomaly detection
- Example: Flag suspicious transactions as they occur

## Architecture

```
Pub/Sub Input Topic
  ↓ Read transactions in real-time
Dataflow Pipeline
  ↓ 1-minute windows
RunInference (PyTorch Model)
  ↓ Generate anomaly scores
Transform Results
  ↓ Extract scores and embeddings
  ├─→ BigQuery (storage & analysis)
  └─→ Pub/Sub Output (downstream processing)
```

## RunInference Benefits

- **In-process**: Model loaded directly in workers (no network calls)
- **Automatic batching**: Combines instances for efficient inference
- **Scalable**: Scales with pipeline workers
- **Real-time**: Low-latency predictions for streaming data
- **Cost-effective**: Pay only when job runs

## Timing Expectations

**Total time from start to results: ~8-10 minutes**

1. **Start Dataflow job**: Instant
2. **Wait for workers**: 3-5 minutes (worker provisioning)
3. **Send test messages**: Instant
4. **Wait for processing**: 2-3 minutes (1-min window + processing)
5. **View results**: Check BigQuery and Pub/Sub

## What This Pipeline Does

1. Read transactions from Pub/Sub input topic
2. Window data into 1-minute batches
3. Format data for PyTorch model
4. Load TorchScript model in workers
5. Run inference to get anomaly scores
6. Extract relevant outputs (score + embeddings)
7. Write results to BigQuery (for analysis)
8. Publish to Pub/Sub output (for downstream systems)
9. Job runs continuously until cancelled

---
## Environment Setup

In [1]:
PROJECT_ID = 'statmike-mlops-349915'
REQ_TYPE = 'ALL'
INSTALL_TOOL = 'poetry'

In [2]:
REQUIREMENTS_URL = 'https://raw.githubusercontent.com/statmike/vertex-ai-mlops/refs/heads/main/Framework%20Workflows/PyTorch/requirements.txt'
REQUIRED_APIS = ["dataflow.googleapis.com", "pubsub.googleapis.com", "bigquery.googleapis.com", "storage.googleapis.com"]

In [3]:
import os, urllib.request
url = 'https://raw.githubusercontent.com/statmike/vertex-ai-mlops/refs/heads/main/core/notebook-template/python_setup.py'
urllib.request.urlretrieve(url, 'python_setup_local.py')
import python_setup_local as python_setup
os.remove('python_setup_local.py')
setup_info = python_setup.setup_environment(PROJECT_ID, REQ_TYPE, REQUIREMENTS_URL, REQUIRED_APIS, INSTALL_TOOL)


PYTHON GCP ENVIRONMENT SETUP

AUTHENTICATION
Checking for existing ADC...
✅ Existing ADC found.
✅ Project is correctly set to 'statmike-mlops-349915'.

API CHECK & ENABLE
✅ dataflow.googleapis.com is already enabled.
✅ pubsub.googleapis.com is already enabled.
✅ bigquery.googleapis.com is already enabled.
✅ storage.googleapis.com is already enabled.

PACKAGE MANAGEMENT
Installation Tool: poetry
✅ Found poetry at: /usr/local/google/home/statmike/.local/bin/poetry
✅ Running in poetry environment: /usr/local/google/home/statmike/.cache/pypoetry/virtualenvs/frameworks-pytorch-0KVJlKeQ-py3.13
ℹ️  Poetry mode: Installing from pyproject.toml (REQUIREMENTS_URL ignored)
✅ Found pyproject.toml at: /usr/local/google/home/statmike/Git/vertex-ai-mlops/Framework Workflows/PyTorch/pyproject.toml
   Changed working directory to: /usr/local/google/home/statmike/Git/vertex-ai-mlops/Framework Workflows/PyTorch
Running poetry install...
   Restored working directory to: /usr/local/google/home/statmike/Gi

---
## Python Setup

In [4]:
import subprocess
import apache_beam as beam
from apache_beam import window
from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.pytorch_inference import PytorchModelHandlerTensor
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
from apache_beam.io.gcp.pubsub import ReadFromPubSub, WriteToPubSub
from apache_beam.io.gcp.bigquery import WriteToBigQuery
import torch
import json
from datetime import datetime
import time

In [5]:
PROJECT_ID = subprocess.run(["gcloud", "config", "get-value", "project"], capture_output=True, text=True, check=True).stdout.strip()

# Configuration
REGION = "us-central1"
SERIES = "frameworks"
EXPERIMENT = "pytorch-autoencoder"

# GCS paths (aligned with dataflow-setup.ipynb)
BUCKET_NAME = PROJECT_ID
BUCKET_URI = f"gs://{BUCKET_NAME}"
MODEL_PATH = f"{BUCKET_URI}/{SERIES}/{EXPERIMENT}/dataflow/final_model_traced.pt"

# Pub/Sub configuration
INPUT_SUB = f"projects/{PROJECT_ID}/subscriptions/{EXPERIMENT}-input-sub"
OUTPUT_TOPIC = f"projects/{PROJECT_ID}/topics/{EXPERIMENT}-output"

# BigQuery configuration
BQ_DATASET = SERIES.replace("-", "_")
BQ_TABLE = f"{EXPERIMENT.replace('-', '_')}_streaming_results"

# Dataflow configuration
DATAFLOW_STAGING = f"{BUCKET_URI}/dataflow/staging"
DATAFLOW_TEMP = f"{BUCKET_URI}/dataflow/temp"

print(f"Project: {PROJECT_ID}")
print(f"Model: {MODEL_PATH}")
print(f"Input subscription: {INPUT_SUB}")
print(f"Output topic: {OUTPUT_TOPIC}")
print(f"Results table: {PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}")

Project: statmike-mlops-349915
Model: gs://statmike-mlops-349915/frameworks/pytorch-autoencoder/dataflow/final_model_traced.pt
Input subscription: projects/statmike-mlops-349915/subscriptions/pytorch-autoencoder-input-sub
Output topic: projects/statmike-mlops-349915/topics/pytorch-autoencoder-output
Results table: statmike-mlops-349915.frameworks.pytorch_autoencoder_streaming_results


---
## Create ModelHandler

The ModelHandler wraps the PyTorch model for use with Apache Beam's RunInference transform. This section explains the key design choices for working with TorchScript models in Dataflow.

In [ ]:
class PyTorchAutoencoderHandler(PytorchModelHandlerTensor):
    """
    Custom ModelHandler for PyTorch autoencoder inference.
    
    Key Design Choices:
    
    1. TorchScript Model Loading:
       - Uses torch_script_model_path (not state_dict_path)
       - TorchScript models (.pt) have architecture embedded
       - No need to provide model_class parameter
    
    2. Batch Processing:
       - Input: List of tensors (one per instance)
       - Stack into single batch tensor with torch.stack()
       - Enables efficient GPU/CPU vectorized operations
    
    3. Output Format:
       - Returns list of dicts (one per instance in batch)
       - Each dict contains tensors for that specific instance
       - PytorchModelHandlerTensor passes these dicts through directly
       - Downstream format_result() function converts tensors to Python types
    
    4. Model Output:
       - Autoencoder returns dict: {"denormalized_MAE": tensor, "encoded": tensor, ...}
       - Extract only needed fields (denormalized_MAE for anomaly score, encoded for embeddings)
       - Keep as tensors in run_inference, convert to Python types later
    """
    
    def run_inference(self, batch, model, inference_args=None):
        """
        Run inference on a batch of input tensors.
        
        Args:
            batch: List of torch.Tensor, each shape (30,) for our autoencoder
            model: Loaded TorchScript model
            inference_args: Optional additional arguments (unused)
            
        Returns:
            List of dicts, one per input, containing model outputs as tensors
        """
        # Stack list of tensors into single batch tensor
        # Input: [tensor(30,), tensor(30,), ...] -> Output: tensor(batch_size, 30)
        batch_tensor = torch.stack(batch)

        # Run model inference without gradient computation
        with torch.no_grad():
            predictions = model(batch_tensor)

        # Convert batch output to list of individual results
        # This matches the batch size and allows proper per-instance processing
        results = []
        for i in range(len(batch)):
            results.append({
                "denormalized_MAE": predictions["denormalized_MAE"][i],  # Scalar tensor
                "encoded": predictions["encoded"][i]  # 1D tensor (embedding)
            })
        return results


# Initialize ModelHandler with TorchScript model from GCS
model_handler = PyTorchAutoencoderHandler(
    torch_script_model_path=MODEL_PATH,  # Path to .pt file in GCS
    device="cpu"  # Run on CPU in Dataflow workers
)

print("✅ ModelHandler created")
print(f"   Model: {MODEL_PATH}")
print(f"   Device: cpu")

---
## Configure Worker Compute Resources

Before building the pipeline, configure the compute resources (machine type and autoscaling) for Dataflow workers. These settings directly impact performance, cost, and latency.

In [ ]:
# Worker Compute Configuration
# =============================
# These settings control the machine type and autoscaling behavior for Dataflow workers.
# Proper configuration ensures efficient resource utilization and cost management.

# Machine Type: n1-standard-4
# - 4 vCPUs, 15 GB memory
# - Suitable for PyTorch inference workloads that need moderate CPU and memory
# - Each worker can handle multiple inference requests concurrently
# - Alternative: n1-standard-2 (smaller), n1-standard-8 (larger), or custom machine types

MACHINE_TYPE = "n1-standard-4"

# Autoscaling for Streaming Pipelines
# - min_workers=2: Ensures pipeline remains responsive even with low traffic
# - max_workers=20: Handles traffic spikes without overwhelming resources
# - Dataflow autoscales based on Pub/Sub backlog and processing latency

MIN_WORKERS = 2
MAX_WORKERS = 20

# Why These Settings for Streaming?
# ----------------------------------
# 1. **Minimum Workers (2)**:
#    - Provides redundancy (if one worker fails, pipeline continues)
#    - Reduces cold start latency when traffic arrives
#    - Maintains low end-to-end latency for real-time processing
#
# 2. **Maximum Workers (20)**:
#    - Allows scaling to handle traffic bursts
#    - Each n1-standard-4 worker processes ~100-500 transactions/sec
#    - 20 workers can handle ~2,000-10,000 transactions/sec
#    - Prevents runaway costs from unlimited scaling
#
# 3. **Machine Type (n1-standard-4)**:
#    - PyTorch model loading requires ~2-4 GB memory per worker
#    - 15 GB allows model + batch processing overhead
#    - 4 vCPUs enable parallel batch inference
#    - Cost-effective for moderate throughput requirements

# When to Adjust These Settings:
# -------------------------------
# - **Higher Traffic**: Increase max_workers (e.g., 50-100)
# - **Lower Latency**: Increase min_workers (e.g., 5-10) to pre-warm capacity
# - **Cost Optimization**: Use smaller machine type (n1-standard-2) if memory permits
# - **Larger Models**: Use n1-standard-8 or n1-highmem-4 for memory-intensive models
# - **CPU-Intensive Models**: Use c2-standard-4 for compute-optimized instances

# GPU Support (Optional)
# ----------------------
# Dataflow supports GPU workers for accelerated inference:
# - Machine type: n1-standard-4 (host machine)
# - GPU type: nvidia-tesla-t4, nvidia-tesla-v100, etc.
# - GPU count: 1-4 per worker
# - Requirements:
#   1. Add --worker_machine_type=n1-standard-4
#   2. Add --worker_gpu_type=nvidia-tesla-t4
#   3. Add --worker_gpu_count=1
#   4. Update model handler to use device="cuda"
#   5. Ensure PyTorch is GPU-enabled in requirements file
#
# Note: GPU workers are significantly more expensive than CPU workers.
# Only use for models where GPU acceleration provides meaningful speedup.
# For small models like this autoencoder, CPU inference is more cost-effective.

print("=" * 60)
print("WORKER COMPUTE CONFIGURATION")
print("=" * 60)
print(f"Machine Type: {MACHINE_TYPE}")
print(f"  - vCPUs: 4")
print(f"  - Memory: 15 GB")
print(f"\nAutoscaling:")
print(f"  - Min Workers: {MIN_WORKERS}")
print(f"  - Max Workers: {MAX_WORKERS}")
print(f"  - Estimated capacity: ~{MIN_WORKERS * 100}-{MAX_WORKERS * 500} transactions/sec")
print(f"\nGPU Support: Not configured (CPU inference)")
print(f"  - To enable GPU: Add --worker_gpu_type and --worker_gpu_count")
print("=" * 60)

In [ ]:
def parse_json(message):
    """
    Parse Pub/Sub message and convert to PyTorch tensor.
    
    Args:
        message: Bytes from Pub/Sub containing JSON with 'features' key
        
    Returns:
        torch.Tensor of shape (30,) containing transaction features
    """
    data = json.loads(message.decode("utf-8"))
    return torch.tensor(data["features"], dtype=torch.float32)


def format_result(element, window=beam.DoFn.WindowParam):
    """
    Format model predictions for output to BigQuery and Pub/Sub.
    
    Args:
        element: Dict returned from run_inference containing tensor outputs
        window: Beam window parameter for extracting window boundaries
        
    Returns:
        Dict with Python types suitable for BigQuery/JSON serialization
        
    Note:
        With our custom ModelHandler, element is the dict we returned from run_inference.
        PytorchModelHandlerTensor passes these dicts through directly without wrapping.
        We convert tensor values to Python types here for serialization.
    """
    return {
        "instance_id": str(hash(str(element["denormalized_MAE"].item()))),
        "anomaly_score": float(element["denormalized_MAE"].item()),
        "encoded": element["encoded"].tolist(),
        "timestamp": datetime.utcnow().isoformat(),
        "window_start": window.start.to_utc_datetime().isoformat(),
        "window_end": window.end.to_utc_datetime().isoformat()
    }


def to_json(element):
    """Convert dict to JSON bytes for Pub/Sub publication."""
    return json.dumps(element).encode("utf-8")


# Create requirements file for Dataflow workers
# Workers need PyTorch installed to load and run the model
import tempfile
requirements_content = "torch>=2.0.0\n"

with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False) as f:
    f.write(requirements_content)
    requirements_file = f.name

# Configure Dataflow pipeline options
options = PipelineOptions([
    f"--project={PROJECT_ID}",
    f"--region={REGION}",
    "--runner=DataflowRunner",  # Run on Google Cloud (not locally)
    f"--temp_location={DATAFLOW_TEMP}",
    f"--staging_location={DATAFLOW_STAGING}",
    f"--requirements_file={requirements_file}",  # Install PyTorch in workers
    f"--job_name=pytorch-streaming-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    "--streaming",  # Enable streaming mode
    "--save_main_session",  # Serialize global imports and variables
    # Worker compute configuration
    f"--machine_type={MACHINE_TYPE}",  # Machine type for workers
    f"--num_workers={MIN_WORKERS}",  # Initial/minimum number of workers
    f"--max_num_workers={MAX_WORKERS}",  # Maximum workers for autoscaling
])

print("✅ Streaming pipeline configured")
print(f"   Job will run in: {REGION}")
print(f"   Staging: {DATAFLOW_STAGING}")
print(f"   Requirements: {requirements_file}")
print(f"   Machine type: {MACHINE_TYPE}")
print(f"   Worker scaling: {MIN_WORKERS}-{MAX_WORKERS} workers")

### Run Streaming Job

Build and execute the pipeline graph. The job will:
1. Read streaming data from Pub/Sub
2. Window into 1-minute fixed windows
3. Run PyTorch model inference via RunInference
4. Write results to both BigQuery and Pub/Sub output topic

The pipeline runs continuously until explicitly cancelled.

In [8]:
p = beam.Pipeline(options=options)

results = (
    p
    | "Read from Pub/Sub" >> ReadFromPubSub(subscription=INPUT_SUB)
    | "Parse JSON" >> beam.Map(parse_json)
    | "Window (1 min)" >> beam.WindowInto(window.FixedWindows(60))
    | "RunInference" >> RunInference(model_handler)
    | "Format results" >> beam.Map(format_result)
)

# Write to Pub/Sub
_ = results | "To JSON" >> beam.Map(to_json) | "Write to Pub/Sub" >> WriteToPubSub(topic=OUTPUT_TOPIC)

# Write to BigQuery
_ = results | "Write to BigQuery" >> WriteToBigQuery(
    table=f"{PROJECT_ID}:{BQ_DATASET}.{BQ_TABLE}",
    write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
)

result = p.run()
print("\n✅ Streaming job started!")
print(f"Monitor: https://console.cloud.google.com/dataflow/jobs/{REGION}?project={PROJECT_ID}")
print("\n" + "=" * 60)
print("⏳ IMPORTANT: Wait 3-5 minutes for Dataflow workers to start")
print("=" * 60)
print("The pipeline needs time to:")
print("  1. Provision workers (2-3 minutes)")
print("  2. Initialize the environment")
print("  3. Connect to Pub/Sub subscription")
print("\nOnce workers are running, you can send test data.")
print("Check the Dataflow console to see when workers are ready.")
print("=" * 60)


✅ Streaming job started!
Monitor: https://console.cloud.google.com/dataflow/jobs/us-central1?project=statmike-mlops-349915

⏳ IMPORTANT: Wait 3-5 minutes for Dataflow workers to start
The pipeline needs time to:
  1. Provision workers (2-3 minutes)
  2. Initialize the environment
  3. Connect to Pub/Sub subscription

Once workers are running, you can send test data.
Check the Dataflow console to see when workers are ready.


---
## Simulate Streaming Data

**⚠️ Wait 3-5 minutes** after starting the Dataflow job before running this cell. Check the [Dataflow Console](https://console.cloud.google.com/dataflow/jobs/us-central1?project=statmike-mlops-349915) to verify workers are running.

In [9]:
time.sleep(5*60)

In [10]:
from google.cloud import pubsub_v1
import time

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(PROJECT_ID, f"{EXPERIMENT}-input")

# Send test messages
for i in range(5):
    message = {"features": [0.1] * 30}  # Dummy transaction
    publisher.publish(topic_path, json.dumps(message).encode("utf-8"))
    print(f"Published message {i+1}")
    time.sleep(2)

print("\n✅ Sent 5 test messages")

Published message 1
Published message 2
Published message 3
Published message 4
Published message 5

✅ Sent 5 test messages


### Monitor Results

**⏳ Wait 2-3 minutes** after sending test messages for the pipeline to process them.

Monitor results from both output destinations: BigQuery (storage/analysis) and Pub/Sub (downstream processing).

In [11]:
time.sleep(120)

In [12]:
from google.cloud import bigquery
from google.cloud import pubsub_v1

# Monitor BigQuery results
print("=" * 60)
print("BIGQUERY RESULTS (Storage & Analysis)")
print("=" * 60)

bq = bigquery.Client(project=PROJECT_ID)
query = f"SELECT * FROM `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}` ORDER BY timestamp DESC LIMIT 10"
df = bq.query(query).to_dataframe()

if len(df) > 0:
    print(f"✅ Found {len(df)} results in BigQuery")
    display(df)
else:
    print("⚠️  No results yet in BigQuery")
    print("   Wait a few minutes for the pipeline to process data")

# Monitor Pub/Sub output topic
print("\n" + "=" * 60)
print("PUB/SUB OUTPUT TOPIC (Downstream Processing)")
print("=" * 60)

subscriber = pubsub_v1.SubscriberClient()
output_sub_path = subscriber.subscription_path(PROJECT_ID, f"{EXPERIMENT}-output-sub")

print(f"Pulling messages from: {output_sub_path}")

# Pull messages from output subscription
try:
    response = subscriber.pull(
        request={"subscription": output_sub_path, "max_messages": 5},
        timeout=10
    )
    
    if response.received_messages:
        print(f"✅ Found {len(response.received_messages)} messages in output topic")
        print("\nSample messages:")
        for i, msg in enumerate(response.received_messages[:3], 1):
            data = json.loads(msg.message.data.decode("utf-8"))
            print(f"\nMessage {i}:")
            print(f"  Anomaly Score: {data.get('anomaly_score', 'N/A')}")
            print(f"  Instance ID: {data.get('instance_id', 'N/A')}")
            print(f"  Timestamp: {data.get('timestamp', 'N/A')}")
        
        # Acknowledge messages (optional - remove if you want to keep them)
        ack_ids = [msg.ack_id for msg in response.received_messages]
        subscriber.acknowledge(request={"subscription": output_sub_path, "ack_ids": ack_ids})
        print(f"\n✅ Acknowledged {len(ack_ids)} messages")
    else:
        print("ℹ️  No messages currently in output subscription")
        print("   Messages may have been consumed already or not yet published")
        
except Exception as e:
    if "DeadlineExceeded" in str(type(e).__name__):
        print("ℹ️  No messages available in output subscription (timeout)")
        print("   This is normal - messages are consumed quickly or not yet available")
    else:
        print(f"⚠️  Error pulling messages: {e}")

print("\n" + "=" * 60)
print("💡 Pipeline Status Summary")
print("=" * 60)

# Get total count from BigQuery
count_query = f"SELECT COUNT(*) as total FROM `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}`"
count_result = bq.query(count_query).to_dataframe()
total_results = count_result['total'].iloc[0] if len(count_result) > 0 else 0

if total_results > 0:
    print(f"✅ Pipeline is working correctly!")
    print(f"   Total results in BigQuery: {total_results}")
    print(f"   Results are being written to both:")
    print(f"     - BigQuery table: {PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}")
    print(f"     - Pub/Sub topic: {OUTPUT_TOPIC}")
else:
    print("⚠️  No results yet in BigQuery")
    print("   Pipeline may still be processing or waiting for data")
    
print("=" * 60)

BIGQUERY RESULTS (Storage & Analysis)
✅ Found 10 results in BigQuery


,instance_id,anomaly_score,encoded,timestamp,window_start,window_end
0,-7797140705466622838,2647.894531,"[0.0, 0.0, 0.4922903776168823, 0.0]",2025-11-07 21:37:32.695437+00:00,2025-11-07 21:25:00+00:00,2025-11-07 21:26:00+00:00
1,-7797140705466622838,2647.894531,"[0.0, 0.0, 0.4922903776168823, 0.0]",2025-11-07 21:37:09.924852+00:00,2025-11-07 21:35:00+00:00,2025-11-07 21:36:00+00:00
2,-7797140705466622838,2647.894531,"[0.0, 0.0, 0.4922903776168823, 0.0]",2025-11-07 21:37:09.910588+00:00,2025-11-07 20:28:00+00:00,2025-11-07 20:29:00+00:00
3,3106348164575065947,2647.894531,"[0.0, 0.0, 0.4922903776168823, 0.0]",2025-11-07 21:37:09.908446+00:00,2025-11-07 21:35:00+00:00,2025-11-07 21:36:00+00:00
4,-7797140705466622838,2647.894531,"[0.0, 0.0, 0.4922903776168823, 0.0]",2025-11-07 21:37:09.905995+00:00,2025-11-07 20:28:00+00:00,2025-11-07 20:29:00+00:00
5,3106348164575065947,2647.894531,"[0.0, 0.0, 0.4922903776168823, 0.0]",2025-11-07 21:37:09.903438+00:00,2025-11-07 20:28:00+00:00,2025-11-07 20:29:00+00:00
6,-7797140705466622838,2647.894531,"[0.0, 0.0, 0.4922903776168823, 0.0]",2025-11-07 21:37:09.903277+00:00,2025-11-07 21:12:00+00:00,2025-11-07 21:13:00+00:00
7,-7797140705466622838,2647.894531,"[0.0, 0.0, 0.4922903776168823, 0.0]",2025-11-07 21:37:09.900754+00:00,2025-11-07 20:28:00+00:00,2025-11-07 20:29:00+00:00
8,-7797140705466622838,2647.894531,"[0.0, 0.0, 0.4922903776168823, 0.0]",2025-11-07 21:37:09.898653+00:00,2025-11-07 21:24:00+00:00,2025-11-07 21:25:00+00:00
9,3106348164575065947,2647.894531,"[0.0, 0.0, 0.4922903776168823, 0.0]",2025-11-07 21:37:09.815809+00:00,2025-11-07 21:12:00+00:00,2025-11-07 21:13:00+00:00



PUB/SUB OUTPUT TOPIC (Downstream Processing)
Pulling messages from: projects/statmike-mlops-349915/subscriptions/pytorch-autoencoder-output-sub
✅ Found 5 messages in output topic

Sample messages:

Message 1:
  Anomaly Score: 2647.89453125
  Instance ID: 3106348164575065947
  Timestamp: 2025-11-07T21:37:09.743365

Message 2:
  Anomaly Score: 2647.89453125
  Instance ID: 3106348164575065947
  Timestamp: 2025-11-07T21:37:09.739753

Message 3:
  Anomaly Score: 2647.89453125
  Instance ID: -7797140705466622838
  Timestamp: 2025-11-07T21:37:09.740229

✅ Acknowledged 5 messages

💡 Pipeline Status Summary
✅ Pipeline is working correctly!
   Total results in BigQuery: 35
   Results are being written to both:
     - BigQuery table: statmike-mlops-349915.frameworks.pytorch_autoencoder_streaming_results
     - Pub/Sub topic: projects/statmike-mlops-349915/topics/pytorch-autoencoder-output


---
## Clean Up

**⚠️ IMPORTANT: Streaming jobs run continuously until explicitly cancelled**, incurring ongoing costs for workers and resources.

### Centralized Cleanup Notebook

For comprehensive cleanup of all Dataflow resources, use the centralized cleanup notebook:

**[dataflow-cleanup.ipynb](./dataflow-cleanup.ipynb)**

This notebook provides:
- ✅ **Stop Dataflow Jobs**: Cancel running streaming/batch jobs created by these notebooks
- ✅ **Clean BigQuery Tables**: Truncate or delete result tables
- ✅ **Clean Pub/Sub Resources**: Delete topics and subscriptions
- ✅ **Clean GCS Files**: Delete model files uploaded for Dataflow
- ✅ **Granular Control**: Use flags to choose exactly what to clean up
- ✅ **Safety Checks**: Warnings for risky operations
- ✅ **Confirmation Prompts**: Review before executing

### Why Use Centralized Cleanup?

- **One location**: Manage all Dataflow infrastructure from a single notebook
- **Comprehensive**: Clean up jobs, tables, Pub/Sub, and GCS files together
- **Flexible**: Truncate tables without deleting schema (useful for testing)
- **Safe**: Built-in safety checks and confirmation prompts
- **Efficient**: Clean up resources from all 4 Dataflow notebooks at once

### Quick Cleanup (This Job Only)

If you only need to stop the current streaming job, monitor it in the [Dataflow Console](https://console.cloud.google.com/dataflow/jobs/us-central1?project=statmike-mlops-349915) and use the "Stop" button.

**Note**: Batch jobs complete automatically and don't require manual cancellation.

---
## Summary

In this notebook, you successfully:

✅ **Created Custom PyTorch ModelHandler**
- Wrapped TorchScript model for RunInference
- Implemented efficient batch processing with torch.stack()
- Designed output format compatible with downstream processing

✅ **Built Streaming Dataflow Pipeline**
- Real-time data ingestion from Pub/Sub
- 1-minute fixed windows for micro-batching
- Automatic dependency installation (PyTorch) in workers

✅ **Deployed PyTorch Model In-Process**
- Model loaded directly in Dataflow workers
- No network calls to external endpoints
- Low-latency inference for streaming data

✅ **Implemented Dual Output Pattern**
- BigQuery: Long-term storage and SQL analytics
- Pub/Sub: Real-time downstream event processing

## Key Learnings

**ModelHandler Design:**
- TorchScript models use `torch_script_model_path` (not `state_dict_path`)
- Batch processing requires `torch.stack()` to combine tensors
- Return list of dicts (one per instance) from `run_inference()`
- Keep outputs as tensors until format stage for efficiency

**Streaming Considerations:**
- Workers need 3-5 minutes to provision and initialize
- Windows control micro-batching granularity (1-minute = balance latency vs. throughput)
- PyTorch installation via `--requirements_file` adds to startup time
- Continuous jobs require manual cancellation to stop charges

**Cost Optimization:**
- In-process inference avoids endpoint API costs
- Pay only while job runs (vs. always-on endpoints)
- Worker autoscaling based on Pub/Sub backlog
- Consider batch processing for historical analysis

## Next Steps

### Continue with Dataflow Workflows:

**Batch Inference with Vertex Endpoint:**
- [dataflow-batch-runinference-vertex.ipynb](./dataflow-batch-runinference-vertex.ipynb)
  - Process historical data via Vertex AI Endpoint
  - Compare performance vs local RunInference
  - Centralized model updates without redeploying pipeline

**Streaming Inference with Vertex Endpoint:**
- [dataflow-streaming-runinference-vertex.ipynb](./dataflow-streaming-runinference-vertex.ipynb)
  - Real-time endpoint calls from Dataflow
  - Shared model across multiple services
  - Managed infrastructure with auto-scaling

**Batch Inference with Local Model:**
- [dataflow-batch-runinference.ipynb](./dataflow-batch-runinference.ipynb)
  - Historical data processing with in-process model
  - Higher throughput for bounded datasets
  - Same ModelHandler pattern as streaming

### Production Enhancements:

1. **Error Handling**: Add dead letter queues for failed predictions
2. **Monitoring**: Set up Cloud Monitoring alerts for job failures and backlog
3. **Autoscaling**: Configure min/max workers based on traffic patterns
4. **Model Versioning**: Implement blue/green deployments for model updates
5. **Data Validation**: Add schema validation for input messages

### Resources

- [Dataflow Streaming Pipelines](https://cloud.google.com/dataflow/docs/concepts/streaming-pipelines)
- [Apache Beam RunInference](https://beam.apache.org/documentation/ml/about-ml/)
- [PyTorch RunInference](https://beam.apache.org/documentation/ml/pytorch-inference/)
- [Pub/Sub Best Practices](https://cloud.google.com/pubsub/docs/publisher)
- [Windowing in Beam](https://beam.apache.org/documentation/programming-guide/#windowing)